# Linearization: Design Study B

In [1]:
import sympy
from sympy import *
from sympy.physics.vector.printing import vlatex
from IPython.display import Math, display

init_printing()

def dotprint(expr):
    display(Math(vlatex(expr)))

In [2]:
t = symbols('t')
# Generalized coordinates
z = symbols('z', cls=Function)

z = z(t)
z_dot = z.diff(t)

m, k, b, F = symbols('m, k, b, F')

In [3]:
LHS = Matrix([
    (m1 + m2)*z_ddot + Rational(1,2)*m1*ell*theta_ddot*cos(theta),
    Rational(1,2)*m1*ell*z_ddot*cos(theta) + Rational(1,3)*m1*ell**2*theta_ddot,
])
RHS = Matrix([
    Rational(1,2)*m1*ell*theta_dot**2*sin(theta) - b*z_dot + F,
    Rational(1,2)*m1*g*ell*sin(theta)
])

dynamics = Eq(LHS, RHS)
dotprint(dynamics)

<IPython.core.display.Math object>

## Deriving Nonlinear State Space Equations

Solve for our highest derivatives, $\ddot{z}, \ddot{\theta}$:

In [4]:
# TODO: Change this!
solve_dict = solve(dynamics, (z_ddot, theta_ddot), simplify=True, dict=True)[0]
dotprint(solve_dict)

<IPython.core.display.Math object>

Create our state vector $x = [x_1, x_2, x_3, x_4]$:

In [8]:
# TODO: Change this!
state = MatrixSymbol('x', 4,1)
dotprint(state)

<IPython.core.display.Math object>

We have $x = [x_1, x_2,x_3,x_4] = [z, \theta, \dot{z}, \dot{\theta}]$, so $\dot{x} = [\dot{z}, \dot{\theta}, \ddot{z}, \ddot{\theta}]$. Let's put that in a vector:

In [10]:
# TODO: Change this!
z_ddot_expr = solve_dict[z_ddot]
theta_ddot_expr = solve_dict[theta_ddot]
state_deriv = Matrix([z_dot, theta_dot, z_ddot_expr, theta_ddot_expr])
dotprint(state_deriv)

<IPython.core.display.Math object>

Now we can substitute in our $x_1, x_2$ values in!

In [18]:
# Dictionary for substitutions
subs_dict = {
    z: state[0],
    theta: state[1],
    z_dot: state[2],
    theta_dot: state[3]

}

f_expr = state_deriv.subs(subs_dict)
dotprint(f_expr)

<IPython.core.display.Math object>

We now have our nonlinear state space equations!

## Linearization

First, we need to find an equilibrium point. We can either do this by hand or use Sympy's `solve` function to do this.

**Note:** It's a good idea to practice doing this by hand in addition to using Sympy!

In [25]:
# TODO: Change this!
# This equation represents f(x,u) = 0
equilibrium_equation = Eq(f_expr, Matrix([0,0,0,0]))

eq_solve_dict = solve(equilibrium_equation, (state[0], state[1], state[2], state[3], F), simplify=True, dict=True)[0]
dotprint(eq_solve_dict)

<IPython.core.display.Math object>

As a sanity check, let's inspect this solution. We know that $x_2, x_3$ have to be zero from the form of $f(x,u)$ above. Let's substitute those values into `f_expr` and see what we get:

In [27]:
dotprint(f_expr.subs({state[2]: 0, state[3]: 0}))

<IPython.core.display.Math object>

We can see that setting $F = 0$ and $x_1 = 0$ causes all numerator terms to go to zero. However, there is another equilibrium solution for $x_1$ that Sympy may not have found. Can you figure out what it is?

(Hint: At what points does $\sin(x_1) = 0$?)

---

Below we define our equilibrium points $x_e = [-, \theta_e, \dot{z}_e, \dot{\theta}_e]$ and $u_e = F_e$. 

_Think for a minute: Why don't we have a $z_e$ equilibrium point?_

In [23]:
#u_eq = eq_solve_dict[]
theta_dot_eq = eq_solve_dict[state[2]]
z_dot_eq = eq_solve_dict[state[3]]

#### Define A, B Jacobians

We can use Sympy's `jacobian` function to find the jacobians of `f(x,u)`.

First we find $A = \frac{\partial f}{\partial x}$:

In [24]:
A = f_expr.jacobian(state)
dotprint(A)

<IPython.core.display.Math object>

We can evaluate at our specific $(x_e, u_e)$ point using `subs`.

In [ ]:
A_subs = { 

    
}

A_eq = A.subs(A_subs)
dotprint(A_eq)

Now we do a similar process to find $B = \frac{\partial f}{\partial u}$

In [ ]:
B = ...
dotprint(B)

Like for the $A$ matrix, we need to substitute in our equilibrium values.

In [ ]:
B_subs = {
    ...
}

B_eq = B.subs(B_subs)
dotprint(B_eq)

### Final Equations

To obtain our final equations of motion, we define new variables measuring our offset from the equilibrium:

$$
\begin{align*}
\tilde{x} &= x - x_e \\
\tilde{u} &= u - u_e
\end{align*}
$$

Our final equations of motion are:

$$
\dot{\tilde{x}} = A\tilde{x} - B\tilde{u}
$$